# tables, (x)arrays, and rasters

Hi all,

I've been playing around with some ideas for working with geospatial raster data. I'd be curious for any feedback you have.

The core question: *what's the best data model for raster data in Python?* Unsurprisingly, I think the answer is "it depends". Let's use work through a concrete task and evaluate the various options. Suppose we wanted to compute NDVI for all the scenes captured by Landsat 8 over some time period.

We'll use the Planetary Computer's STAC API to find the scenes, and geopandas to plot them on a map.

In [1]:
import warnings

warnings.simplefilter("ignore", FutureWarning)

In [2]:
import pystac_client
import geopandas
import planetary_computer
import pystac
import pandas as pd

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)

items = catalog.search(
    collections=["landsat-8-c2-l2"],
    datetime="2021-07-01T08:00:00Z/2021-07-01T10:00:00Z"
).get_all_items()


items = [planetary_computer.sign(item) for item in items]
items = pystac.ItemCollection(items, clone_items=False)
df = geopandas.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")

# https://github.com/geopandas/geopandas/issues/1208
df["id"] = [x.id for x in items]
m = df[["geometry", "id", "datetime"]].explore()
m

This type of data *can* be represented as an xarray DataArray. But it's not the most efficient way to store the data:

In [3]:
import stackstac

ds = stackstac.stack(
    [x.to_dict() for x in items],
    assets=["SR_B2", "SR_B3", "SR_B4", "SR_B5"],
    epsg=32631,
    chunksize=(7691, 7531)
)
ds

,Array,Chunk
Bytes,426.12 TiB,441.90 MiB
Shape,"(117, 4, 802756, 155889)","(1, 1, 7691, 7531)"
Count,1035081 Tasks,1031940 Chunks
Type,float64,numpy.ndarray


To build this `(time, band, y, x)` DataArray, we end up with many missing values. If you think about the Datacube literally, with some "volume" of observed pixels, we have a lot of empty space. In this case, the DataArray takes 426 TiB to store.

Even if we collapse the time dimension, we still have empty space in the "corners"

In [4]:
ds2 = stackstac.mosaic(ds)
ds2

,Array,Chunk
Bytes,3.64 TiB,441.90 MiB
Shape,"(4, 802756, 155889)","(1, 7691, 7531)"
Count,4113261 Tasks,8820 Chunks
Type,float64,numpy.ndarray


This helps a lot, getting us down to 3.6 TiB (the curse of dimensionality works in reverse too!) But it's still not as efficienty as possible because of that empty space in the corners for this dataset. To actually load all these rasters into, say, a list would take much less memory.

In [5]:
import dask
import math

assets = ["SR_B2", "SR_B3", "SR_B4", "SR_B5"]

nbytes = 0
for item in items:
    for asset in assets:
        nbytes += 8 * math.prod(item.assets[asset].extra_fields["proj:shape"])
        
        
dask.utils.format_bytes(nbytes)

'218.57 GiB'

So *for this dataset* (I cannot emphasize that enough; this example was deliberatly designed to look bad for DataArrays) it doesn't make sense to model the data as a DataArray.

| data model | memory (TiB) |
| ---------- | ------ |
| xarray `(time, band, y, x)` | 426 |
| xarray `(band, y, x)` | 3.6 |
| list       | 0.2 |

I've haven't really considered an `xarray.Dataset` here. I suspect that the memory usage could get down to approximately what would be required by a list of rasters. That said, something like the following seems to cause some issues.

```python
import xarray as xr

arrays = {item.id: stackstac.stack(item.to_dict(), assets=assets, chunksize=-1) for item in items}
ds3 = xr.Dataset(arrays)
```

This causes warnings from Dask about slicing an array producing many chunks. I haven't looked into why. I have a feeling that this would be a bit "untidy", but I haven't worked with Datasets too much.

In the Python data science space, we're fortunate to have both xarray and pandas (and geopandas and dask.dataframe). So we have choices! pandas provides an [extension array interface](https://pandas.pydata.org/docs/development/extending.html#extension-types) to store non-NumPy arrays inside a pandas DataFrame. What would it look like to store STAC items (and more interestingly, rasters stored as DataArrays) inside a pandas DataFrame? Here's a prototype:

Let's load those STAC items into a an "ItemArray". Nothing fancy here.

In [6]:
import rasterpandas

sa = rasterpandas.ItemArray(items)
sa

<ItemArray>
[<Item id=LC08_L2SP_191047_20210701_02_T1>,
 <Item id=LC08_L2SP_191046_20210701_02_T1>,
 <Item id=LC08_L2SP_191045_20210701_02_T1>,
 <Item id=LC08_L2SP_191044_20210701_02_T1>,
 <Item id=LC08_L2SP_191043_20210701_02_T1>,
 <Item id=LC08_L2SP_191042_20210701_02_T1>,
 <Item id=LC08_L2SP_191041_20210701_02_T1>,
 <Item id=LC08_L2SP_191040_20210701_02_T1>,
 <Item id=LC08_L2SP_191039_20210701_02_T1>,
 <Item id=LC08_L2SP_191038_20210701_02_T1>,
 ...
 <Item id=LC08_L2SP_175015_20210701_02_T1>,
 <Item id=LC08_L2SP_175014_20210701_02_T1>,
 <Item id=LC08_L2SP_175013_20210701_02_T2>,
 <Item id=LC08_L2SP_175012_20210701_02_T2>,
 <Item id=LC08_L2SP_175011_20210701_02_T2>,
 <Item id=LC08_L2SP_175010_20210701_02_T1>,
 <Item id=LC08_L2SP_175006_20210701_02_T1>,
 <Item id=LC08_L2SP_175005_20210701_02_T2>,
 <Item id=LC08_L2SP_175001_20210701_02_T1>,
 <Item id=LC08_L2SR_159248_20210701_02_T2>]
Length: 117, dtype: stac

That `ItemArray` can be put inside a pandas Series:

In [7]:
series = pd.Series(sa, name="stac_items")
series

0      <Item id=LC08_L2SP_191047_20210701_02_T1>
1      <Item id=LC08_L2SP_191046_20210701_02_T1>
2      <Item id=LC08_L2SP_191045_20210701_02_T1>
3      <Item id=LC08_L2SP_191044_20210701_02_T1>
4      <Item id=LC08_L2SP_191043_20210701_02_T1>
                         ...                    
112    <Item id=LC08_L2SP_175010_20210701_02_T1>
113    <Item id=LC08_L2SP_175006_20210701_02_T1>
114    <Item id=LC08_L2SP_175005_20210701_02_T2>
115    <Item id=LC08_L2SP_175001_20210701_02_T1>
116    <Item id=LC08_L2SR_159248_20210701_02_T2>
Name: stac_items, Length: 117, dtype: stac

Pandas lets you register accessors. For example, we could have a `stac` accessor that knows how to do stuff with STAC metadata, for example adding a column for each asset in the collection.

In [8]:
rdf = series[:10].stac.with_rasters(assets=["SR_B2", "SR_B3", "SR_B4", "SR_B5"])
rdf

,stac_items,SR_B2,SR_B3,SR_B4,SR_B5
0,<Item id=LC08_L2SP_191047_20210701_02_T1>,"<xarray.DataArray 'stackstac-3e7a1039fae760fc605736b4ac2546cc' (y: 7692, x: 7532)> dask.array<getitem, shape=(7692, 7532), dtype=float64, chunksize=(7692, 7532), chunktype=numpy.ndarray> Coordinates: time datetime64[ns] 2021-07-01T09:59:38.868875 id <U31 'LC08_L2SP_191047_20210701_02_T1' band <U5 'SR_B2' * x (x) float64 6.981e+05 6.981e+05 ... 9.24e+05 * y (y) float64 2.195e+06 2.195e+06 ... 1.964e+06 view:off_nadir int64 0 instruments object {'tirs', 'oli'} landsat:scene_id <U21 'LC81910472021182LGN00' landsat:wrs_path <U3 '191' view:sun_azimuth float64 75.25 landsat:processing_level <U4 'L2SP' proj:epsg int64 32631 landsat:collection_category <U2 'T1' description <U55 'Collection 2 Level-2 Blue Band (B2) Su... view:sun_elevation float64 66.19 landsat:collection_number <U2 '02' eo:cloud_cover float64 0.0 landsat:wrs_row <U3 '047' landsat:wrs_type <U1 '2' proj:bbox object {2194815.0, 1964085.0, 698085.0, 9240... landsat:cloud_cover_land float64 0.0 platform <U9 'landsat-8' gsd float64 30.0 title <U14 'Blue Band (B2)' proj:shape object {7691, 7531} proj:transform object {0.0, -30.0, 698085.0, 30.0, 2194815.0} common_name <U4 'blue' center_wavelength float64 0.48 full_width_half_max float64 0.06 epsg int64 32631 Attributes: spec: RasterSpec(epsg=32631, bounds=(698070.0, 1964070.0, 924030.0... crs: epsg:32631 transform: | 30.00, 0.00, 698070.00|\n| 0.00,-30.00, 2194830.00|\n| 0.0... resolution: 30.0","<xarray.DataArray 'stackstac-0c0d7972688bbdd9170945d63d2e85e5' (y: 7692, x: 7532)> dask.array<getitem, shape=(7692, 7532), dtype=float64, chunksize=(7692, 7532), chunktype=numpy.ndarray> Coordinates: time datetime64[ns] 2021-07-01T09:59:38.868875 id <U31 'LC08_L2SP_191047_20210701_02_T1' band <U5 'SR_B3' * x (x) float64 6.981e+05 6.981e+05 ... 9.24e+05 * y (y) float64 2.195e+06 2.195e+06 ... 1.964e+06 view:off_nadir int64 0 instruments object {'tirs', 'oli'} landsat:scene_id <U21 'LC81910472021182LGN00' landsat:wrs_path <U3 '191' view:sun_azimuth float64 75.25 landsat:processing_level <U4 'L2SP' proj:epsg int64 32631 landsat:collection_category <U2 'T1' description <U56 'Collection 2 Level-2 Green Band (B3) S... view:sun_elevation float64 66.19 landsat:collection_number <U2 '02' eo:cloud_cover float64 0.0 landsat:wrs_row <U3 '047' landsat:wrs_type <U1 '2' proj:bbox object {2194815.0, 1964085.0, 698085.0, 9240... landsat:cloud_cover_land float64 0.0 platform <U9 'landsat-8' gsd float64 30.0 title <U15 'Green Band (B3)' proj:shape object {7691, 7531} proj:transform object {0.0, -30.0, 698085.0, 30.0, 2194815.0} common_name <U5 'green' center_wavelength float64 0.56 full_width_half_max float64 0.06 epsg int64 32631 Attributes: spec: RasterSpec(epsg=32631, bounds=(698070.0, 1964070.0, 924030.0... crs: epsg:32631 transform: | 30.00, 0.00, 698070.00|\n| 0.00,-30.00, 2194830.00|\n| 0.0... resolution: 30.0","<xarray.DataArray 'stackstac-4049c24192117122fa2cab6931bb3ff0' (y: 7692, x: 7532)> dask.array<getitem, shape=(7692, 7532), dtype=float64, chunksize=(7692, 7532), chunktype=numpy.ndarray> Coordinates: time datetime64[ns] 2021-07-01T09:59:38.868875 id <U31 'LC08_L2SP_191047_20210701_02_T1' band <U5 'SR_B4' * x (x) float64 6.981e+05 6.981e+05 ... 9.24e+05 * y (y) float64 2.195e+06 2.195e+06 ... 1.964e+06 view:off_nadir int64 0 instruments object {'tirs', 'oli'} landsat:scene_id <U21 'LC81910472021182LGN00' landsat:wrs_path <U3 '191' view:sun_azimuth float64 75.25 landsat:processing_level <U4 'L2SP' proj:epsg int64 32631 landsat:collection_category <U2 'T1' description <U54 'Collection 2 Level-2 Red Band (B4) Sur... view:sun_elevation float64 66.19 landsat:collection_number <U2 '02' eo:cloud_cover float64 0.0 landsat:wrs_row <U3 '047' landsat:wrs_type <U1 '2' proj:bbox object {2194815.0, 1964085.0, 698085.0, 9240... landsat:cloud_cover_land float64 0.0 platform <U9 'landsat-8' gsd float64 30.0 title <U13 'Red Band (B4)' proj:shape object {7691, 7531} proj:trans

Now things are getting more interesting! The repr is a bit messy, but this new DataFrame has a column for each of the blue, green, red, and nir bands. Each of those is a column of rasters. And each raster is just an xarray.DataArray!

In [9]:
rdf.iloc[1, 1]

<xarray.DataArray 'stackstac-19c64a55a2e4be3447b28ad04a3d22bd' (y: 7852, x: 7702)>
dask.array<getitem, shape=(7852, 7702), dtype=float64, chunksize=(7852, 7702), chunktype=numpy.ndarray>
Coordinates:
    time                         datetime64[ns] 2021-07-01T09:59:14.960892
    id                           <U31 'LC08_L2SP_191046_20210701_02_T1'
    band                         <U5 'SR_B2'
  * x                            (x) float64 1.008e+05 1.008e+05 ... 3.318e+05
  * y                            (y) float64 2.357e+06 2.357e+06 ... 2.121e+06
    view:off_nadir               int64 0
    instruments                  object {'tirs', 'oli'}
    landsat:scene_id             <U21 'LC81910462021182LGN00'
    landsat:wrs_path             <U3 '191'
    view:sun_azimuth             float64 78.44
    landsat:processing_level     <U4 'L2SP'
    proj:epsg                    int64 32632
    landsat:collection_category  <U2 'T1'
    description                  <U55 'Collection 2 Level-2 Blue Band (B2) Su...
    view:sun_elevation           float64 66.73
    landsat:collection_number    <U2 '02'
    eo:cloud_cover               float64 0.0
    landsat:wrs_row              <U3 '046'
    landsat:wrs_type             <U1 '2'
    proj:bbox                    object {100785.0, 2121285.0, 2356815.0, 3318...
    landsat:cloud_cover_land     float64 0.0
    platform                     <U9 'landsat-8'
    gsd                          float64 30.0
    title                        <U14 'Blue Band (B2)'
    proj:shape                   object {7851, 7701}
    proj:transform               object {0.0, -30.0, 2356815.0, 100785.0, 30.0}
    common_name                  <U4 'blue'
    center_wavelength            float64 0.48
    full_width_half_max          float64 0.06
    epsg                         int64 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(100770.0, 2121270.0, 331830.0...
    crs:         epsg:32632
    transform:   | 30.00, 0.00, 100770.00|\n| 0.00,-30.00, 2356830.00|\n| 0.0...
    resolution:  30.0

Then we can have fun with operations. For example, computing NDVI on two columns:

In [10]:
ndvi = rdf.raster.ndvi("SR_B4", "SR_B5")
ndvi[:2]

0   <xarray.DataArray 'ndvi' (y: 7692, x: 7532)>
dask.array<_normalized_ratio_cpu, shape=(7692, 7532), dtype=float64, chunksize=(7692, 7532), chunktype=numpy.ndarray>
Coordinates:
    time                         datetime64[ns] 2021-07-01T09:59:38.868875
    id                           <U31 'LC08_L2SP_191047_20210701_02_T1'
    band                         <U5 'SR_B4'
  * x                            (x) float64 6.981e+05 6.981e+05 ... 9.24e+05
  * y                            (y) float64 2.195e+06 2.195e+06 ... 1.964e+06
    view:off_nadir               int64 0
    instruments                  object {'tirs', 'oli'}
    landsat:scene_id             <U21 'LC81910472021182LGN00'
    landsat:wrs_path             <U3 '191'
    view:sun_azimuth             float64 75.25
    landsat:processing_level     <U4 'L2SP'
    proj:epsg                    int64 32631
    landsat:collection_category  <U2 'T1'
    description                  <U54 'Collection 2 Level-2 Red Band (B4) Sur...
    view:s

That returned a pandas Series. Each element is again a raster:

In [11]:
ndvi.iloc[0]

<xarray.DataArray 'ndvi' (y: 7692, x: 7532)>
dask.array<_normalized_ratio_cpu, shape=(7692, 7532), dtype=float64, chunksize=(7692, 7532), chunktype=numpy.ndarray>
Coordinates:
    time                         datetime64[ns] 2021-07-01T09:59:38.868875
    id                           <U31 'LC08_L2SP_191047_20210701_02_T1'
    band                         <U5 'SR_B4'
  * x                            (x) float64 6.981e+05 6.981e+05 ... 9.24e+05
  * y                            (y) float64 2.195e+06 2.195e+06 ... 1.964e+06
    view:off_nadir               int64 0
    instruments                  object {'tirs', 'oli'}
    landsat:scene_id             <U21 'LC81910472021182LGN00'
    landsat:wrs_path             <U3 '191'
    view:sun_azimuth             float64 75.25
    landsat:processing_level     <U4 'L2SP'
    proj:epsg                    int64 32631
    landsat:collection_category  <U2 'T1'
    description                  <U54 'Collection 2 Level-2 Red Band (B4) Sur...
    view:sun_elevation           float64 66.19
    landsat:collection_number    <U2 '02'
    eo:cloud_cover               float64 0.0
    landsat:wrs_row              <U3 '047'
    landsat:wrs_type             <U1 '2'
    proj:bbox                    object {2194815.0, 1964085.0, 698085.0, 9240...
    landsat:cloud_cover_land     float64 0.0
    platform                     <U9 'landsat-8'
    gsd                          float64 30.0
    title                        <U13 'Red Band (B4)'
    proj:shape                   object {7691, 7531}
    proj:transform               object {0.0, -30.0, 698085.0, 30.0, 2194815.0}
    common_name                  <U3 'red'
    center_wavelength            float64 0.65
    full_width_half_max          float64 0.04
    epsg                         int64 32631
Attributes:
    spec:        RasterSpec(epsg=32631, bounds=(698070.0, 1964070.0, 924030.0...
    crs:         epsg:32631
    transform:   | 30.00, 0.00, 698070.00|\n| 0.00,-30.00, 2194830.00|\n| 0.0...
    resolution:  30.0

So that's the prototype. It's a fun demonstration of pandas' extension arrays. Is it useful? Maybe. The Spark / Scala world have found that model useful, as implemented by [rasterframes](https://rasterframes.io/). We have xarray, which lowers the *need* for something like this. But perhaps it's still useful.

To reiterate the motivations, and help shed light on when you might chose one vs. the other: If your data is actually a cube, you should probably just use xarray. Doing operations across axes (like time) is probably going to be more efficient than what pandas could provide, and the xarray API will likely be more ergonomic. But if you don't have an actual cube (or if making a cube generates a lot of "empty" space / missing values) then this approach might be worth exploring.